In [1]:
import pandas as pd
import numpy as np
#import tensorflow as tf

## Loading the data

In [2]:
data = pd.read_csv('C:/Users/LENOVO/Desktop/Integration/Absenteeism_Preprocessed.csv')

In [3]:
data.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [4]:
# Creating target variables

data['Absenteeism Time in Hours'].median()

3.0

In [5]:
targets = np.where(data['Absenteeism Time in Hours'] > 3, 1, 0)

In [6]:
data['Absenteeism'] = targets

In [7]:
data.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [8]:
#let's see do we have to balance the data

targets.sum()/targets.shape[0]

0.45571428571428574

 46% of the targets are 1's therefore, it's alright

In [9]:
data.drop(['Absenteeism Time in Hours'], axis=1, inplace=True)
data.drop(['Day of the Week', 'Distance to Work', 'Daily Work Load Average'], axis=1, inplace=True)

In [10]:
#Selecting inputs

unscaled_inputs = data.iloc[:,:-1]

### Standardizing the data

In [11]:
from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin

#this is done as we didnt want to standardize the dummy variables

class CustomScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns, copy=True,with_mean=True, with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [13]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [14]:
#columns_to_scale = ['Month Value', 'Day of the Week', 'Transportation Expense', 'Distance to Work',
      # 'Age', 'Daily Work Load Average', 'Body Mass Index', 'Children', 'Pets']
columns_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Education']
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_omit]

In [15]:
scaler = CustomScaler(columns_to_scale)

C:\Users\LENOVO\Anaconda3\envs\py3-TF2.0\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass copy=True, with_mean=True, with_std=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [16]:
scaler.fit(unscaled_inputs)
scaled_inputs = scaler.transform(unscaled_inputs)

In [17]:
#scaled_inputs = scaler.fit_transform(unscaled_inputs)

In [18]:
scaled_inputs.shape

(700, 11)

In [19]:
#splitting the data

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, test_size=0.2, random_state=20)

In [20]:
print(x_train.shape)

(560, 11)


### Modelling

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [22]:
reg = LogisticRegression()
reg.fit(x_train, y_train)

LogisticRegression()

In [23]:
reg.score(x_train, y_train)

0.7446428571428572

In [24]:
#!pip install xgboost
#!pip install lazypredict
#import xgboost as xgb
#from lazypredict.Supervised import LazyRegressor

In [25]:
#!pip install lightgbm

In [26]:
#!pip install pytest

In [27]:
#clf = LazyRegressor(ignore_warnings=True)
#models, predictions = clf.fit(x_train, x_test, y_train, y_test)

In [28]:
#models

## Manually building a scoring method

In [29]:
#from sklearn.linear_model import LinearRegression
#lr = LinearRegression()
#lr.fit(x_train, y_train)
#lr.score(x_test, y_test)

In [30]:
#from lazypredict.Supervised import LazyClassifier
#lc = LazyClassifier(ignore_warnings=True)
#model1, predictions1 = lc.fit(x_train, x_test, y_train, y_test)

In [31]:
#model1

In [32]:
model_outputs = reg.predict(x_train)

In [33]:
model_outputs == y_train

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True, False, False, False,  True,  True,
       False,  True, False,  True,  True,  True,  True, False,  True,
       False, False, False, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True, False,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,

In [34]:
np.sum(model_outputs == y_train)

417

this is the total number of true

In [35]:
np.sum(model_outputs==y_train)/model_outputs.shape[0]

0.7446428571428572

we get the same accuracy

In [36]:
#lets get the model line equation

feature_names = unscaled_inputs.columns.values
summar = pd.DataFrame(columns=['Feature Name'], data=feature_names)
summar['Coeff'] = np.transpose(reg.coef_)
summar

,Feature Name,Coeff
0,Reason_1,1.628704
1,Reason_2,-0.081166
2,Reason_3,1.983405
3,Reason_4,-0.637028
4,Month Value,0.165892
5,Transportation Expense,0.572845
6,Age,-0.151663
7,Body Mass Index,0.265399
8,Education,-0.267275
9,Children,0.313401


In [37]:
summar.index = summar.index + 1
summar.loc[0] = ['Intercept', reg.intercept_[0]]
summar = summar.sort_index()
summar

,Feature Name,Coeff
0,Intercept,-0.339433
1,Reason_1,1.628704
2,Reason_2,-0.081166
3,Reason_3,1.983405
4,Reason_4,-0.637028
5,Month Value,0.165892
6,Transportation Expense,0.572845
7,Age,-0.151663
8,Body Mass Index,0.265399
9,Education,-0.267275


In [38]:
summar['Order Ratios'] = np.exp(summar.Coeff)

In [39]:
summar

,Feature Name,Coeff,Order Ratios
0,Intercept,-0.339433,0.712174
1,Reason_1,1.628704,5.097266
2,Reason_2,-0.081166,0.922041
3,Reason_3,1.983405,7.267443
4,Reason_4,-0.637028,0.528862
5,Month Value,0.165892,1.180446
6,Transportation Expense,0.572845,1.773305
7,Age,-0.151663,0.859278
8,Body Mass Index,0.265399,1.303951
9,Education,-0.267275,0.765462


In [40]:
summar.sort_values('Order Ratios', ascending=False)

,Feature Name,Coeff,Order Ratios
3,Reason_3,1.983405,7.267443
1,Reason_1,1.628704,5.097266
6,Transportation Expense,0.572845,1.773305
10,Children,0.313401,1.368070
8,Body Mass Index,0.265399,1.303951
5,Month Value,0.165892,1.180446
2,Reason_2,-0.081166,0.922041
7,Age,-0.151663,0.859278
9,Education,-0.267275,0.765462
11,Pets,-0.293380,0.745739


### Testing the model

In [41]:
reg.score(x_test, y_test)

0.7142857142857143

In [42]:
prob = reg.predict_proba(x_test)

In [43]:
prob

array([[0.73458254, 0.26541746],
       [0.63098132, 0.36901868],
       [0.48348523, 0.51651477],
       [0.80169307, 0.19830693],
       [0.0805696 , 0.9194304 ],
       [0.29549926, 0.70450074],
       [0.27897075, 0.72102925],
       [0.11630889, 0.88369111],
       [0.81973894, 0.18026106],
       [0.76984818, 0.23015182],
       [0.41077533, 0.58922467],
       [0.17251172, 0.82748828],
       [0.06615924, 0.93384076],
       [0.64882908, 0.35117092],
       [0.26573069, 0.73426931],
       [0.61018454, 0.38981546],
       [0.62552984, 0.37447016],
       [0.45727806, 0.54272194],
       [0.36193754, 0.63806246],
       [0.04882551, 0.95117449],
       [0.73903145, 0.26096855],
       [0.80169307, 0.19830693],
       [0.36425731, 0.63574269],
       [0.36425731, 0.63574269],
       [0.22305607, 0.77694393],
       [0.78206827, 0.21793173],
       [0.55965681, 0.44034319],
       [0.87421302, 0.12578698],
       [0.16764163, 0.83235837],
       [0.80169307, 0.19830693],
       [0.

In [44]:
#the probablity of excessive abenteeism

prob[:,1]

array([0.26541746, 0.36901868, 0.51651477, 0.19830693, 0.9194304 ,
       0.70450074, 0.72102925, 0.88369111, 0.18026106, 0.23015182,
       0.58922467, 0.82748828, 0.93384076, 0.35117092, 0.73426931,
       0.38981546, 0.37447016, 0.54272194, 0.63806246, 0.95117449,
       0.26096855, 0.19830693, 0.63574269, 0.63574269, 0.77694393,
       0.21793173, 0.44034319, 0.12578698, 0.83235837, 0.19830693,
       0.47780054, 0.69970236, 0.72383218, 0.61194349, 0.19830693,
       0.62536914, 0.18736616, 0.7695765 , 0.35604426, 0.54212492,
       0.19088441, 0.48702473, 0.20221817, 0.69124122, 0.84529199,
       0.6301178 , 0.72029978, 0.26541746, 0.19965198, 0.18367605,
       0.61516844, 0.62726185, 0.70450074, 0.24893743, 0.52512867,
       0.39133569, 0.88695238, 0.20308108, 0.64757202, 0.65830451,
       0.72083267, 0.69454598, 0.26199983, 0.81529117, 0.17465633,
       0.24736297, 0.27564046, 0.18736616, 0.7741618 , 0.39545465,
       0.18736616, 0.6032739 , 0.91522582, 0.41198747, 0.53980

### Saving the module

In [45]:
import pickle

In [46]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [47]:
with open('scaler', 'wb') as file:
    pickle.dump(scaler, file)